In [2]:
a = """
Develop program which will take as an input a path to txt file with some content. This file should be read and content transformed into pre-defined json structure using LLM API. 
This json structure should be defined as orginized form for deck content. For instance each slide should have header, then some content - text blocks or lists or both. So there might be some strictly defined fields for each slide which should be filled 
according to the provided content. develop this json structure.
Then write another program which should take this json and using predefined functions for python-pptx library - generate the deck slide by slide from content definitions in json file. Develop end-to-end pipleine for comprehensive deck structures.
"""

In [8]:
import json

def transform_content_with_llm(text_content):
    # Placeholder function to simulate LLM API call
    # In a real scenario, this function would send the text content to an LLM API
    # and receive the transformed JSON structure as a response.
    # For demonstration purposes, we'll return a hardcoded JSON structure.
    return {
        "slides": [
            {
                "layout": "Title and Content",
                "title": "Slide 1 Title",
                "content": [
                    {
                        "type": "text",
                        "text": "This is a text block."
                    },
                    {
                        "type": "list",
                        "items": ["Item 1", "Item 2", "Item 3"]
                    },
                    {
                        "type": "image",
                        "path": "corrective_rag_graph.png",
                        "position": {"left": 1, "top": 1, "width": 2, "height": 2}
                    },
                    {
                        "type": "shape",
                        "shape_type": "ROUNDED_RECTANGLE",
                        "position": {"left": 1, "top": 3, "width": 2, "height": 1},
                        "text": "This is a shape with text."
                    },
                    {
                        "type": "table",
                        "rows": 2,
                        "columns": 2,
                        "data": [
                            ["Header 1", "Header 2"],
                            ["Row 1 Col 1", "Row 1 Col 2"]
                        ],
                        "position": {"left": 1, "top": 4, "width": 6, "height": 1}
                    }
                ]
            },
            {
                "layout": "Title Only",
                "title": "Slide 2 Title",
                "content": [
                    {
                        "type": "text",
                        "text": "This is another text block."
                    }
                ]
            }
        ]
    }

def read_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def main(file_path):
    # text_content = read_text_file(file_path)
    json_structure = transform_content_with_llm("")
    with open('deck_content.json', 'w') as json_file:
        json.dump(json_structure, json_file, indent=4)

main("")
# json_file_path = 'deck_content.json'
# output_pptx_path = 'output_presentation.pptx'
# main(json_file_path, output_pptx_path)

In [26]:
import json
from pptx import Presentation
from pptx.util import Inches, Pt

def load_json_from_disk(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def create_presentation_from_json(json_structure, output_path):
    prs = Presentation()

    styles = json_structure.get('styles', {})

    for slide_data in json_structure['slides']:
        slide_layout = prs.slide_layouts[1]  # Title and Content layout
        slide = prs.slides.add_slide(slide_layout)

        title = slide.shapes.title
        title.text = slide_data['title']
        title_font_size = styles.get('title', {}).get('font_size', 24)
        title.text_frame.paragraphs[0].font.size = Pt(title_font_size)

        for content in slide_data['content']:
            if content['type'] == 'text':
                text_frame = slide.shapes.placeholders[1].text_frame
                p = text_frame.add_paragraph()
                p.text = content['text']
                text_block_font_size = styles.get(content.get('style', 'text_block'), {}).get('font_size', 18)
                p.font.size = Pt(text_block_font_size)
            elif content['type'] == 'list':
                text_frame = slide.shapes.placeholders[1].text_frame
                for item in content['items']:
                    p = text_frame.add_paragraph()
                    p.text = item
                    p.level = 1
                    list_item_font_size = styles.get(content.get('style', 'list_item'), {}).get('font_size', 16)
                    p.font.size = Pt(list_item_font_size)
            elif content['type'] == 'image':
                left = Inches(content['position']['left'])
                top = Inches(content['position']['top'])
                width = Inches(content['position']['width'])
                height = Inches(content['position']['height'])
                slide.shapes.add_picture(content['path'], left, top, width, height)
            elif content['type'] == 'table':
                left = Inches(content['position']['left'])
                top = Inches(content['position']['top'])
                width = Inches(content['position']['width'])
                height = Inches(content['position']['height'])
                rows = content['rows']
                cols = content['columns']
                table = slide.shapes.add_table(rows, cols, left, top, width, height).table
                for i, row in enumerate(content['data']):
                    for j, cell in enumerate(row):
                        table.cell(i, j).text = cell

    prs.save(output_path)


In [23]:
with open("deck_content.json", 'r') as file:
    json_structure = json.load(file)

In [27]:
json_data = load_json_from_disk("deck_content.json")
create_presentation_from_json(json_structure, "output_presentation.pptx")